# COGS 108 - Final Project

## Important

- ONE, and only one, member of your group should upload this notebook to TritonED. 
- Each member of the group will receive the same grade on this assignment. 
- Keep the file name the same: submit the file 'FinalProject.ipynb'.
- Only upload the .ipynb file to TED, do not upload any associted data. Make sure that for cells in which you want graders to see output that these cells have been executed.

## Group Members: Fill in the Student IDs of each group member here

Replace the lines below to list each persons full student ID, ucsd email and full name.

-  
- 
- 
- 



# Part 0 - Importing the required packages

In [1]:
import pandas as pd # for use of dataframes
import numpy as np
import matplotlib.pyplot as plt # for plotting graphs 
import statsmodels.api as sm # for fitting an OLS model
import scipy.stats as stats # for access to tests for normal distribution
from scipy.stats import ttest_ind, normaltest # for access to t tests, normal test

# Part 1 - Introduction and Background

Chicago, otherwise known as 'The Windy City', has some of the worst crime rates in America. Some of this has to do with statitical flukes; it is the third largest city in the U.S., which makes any type of criminal activity more easily reportable. However, most people wouldn't argue over the real danger you are in when you go to some of the worse off areas of Chicago.

We wanted to analyze the crime rates in Chicago in some way in order to assist people in the city in understanding what problems exist and where they are happening. In the end, we decided to ask the following question:

"Does the district a crime is committed in or type of crime more closely predict whether an arrest was made".

The answers derived from the answer to this question could spawn further analysis, e.g.:

1. If a particular crime is more likely to result in an arrest in one district over the other, why is that? Is it the police in that district? Is it the citizens? Or is it the geography?

2. If the type of crime affects whether an arrest is made, does that mean that we are more afraid of certain crimes over others? Is this a valid fear? 

# Part 2 - Data Description

We are using this dataset: https://www.kaggle.com/chicago/chicago-crime

This is chicago crime data from 2001 to present. It represents all reported instances of crime, and whether or not an arrest has been made since the data was last released. Each row represents a crime, except for multiple homicides, for which there is a different row for each victim of the crime. 

Here are some basic stats:

In [2]:
crime = pd.read_csv('~/data/CrimesSmall.csv')

In [3]:
crime.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,...,28.0,25.0,04B,1144606.0,1903566.0,2015,02/10/2018 03:50:01 PM,41.891399,-87.744385,"(41.891398861, -87.744384567)"
1,10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,False,...,15.0,67.0,26,1166468.0,1860715.0,2015,02/10/2018 03:50:01 PM,41.773372,-87.665319,"(41.773371528, -87.665319468)"
2,10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,4.0,39.0,08B,1185075.0,1875622.0,2015,02/10/2018 03:50:01 PM,41.813861,-87.596643,"(41.81386068, -87.596642837)"
3,10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,False,False,...,3.0,40.0,08B,1178033.0,1870804.0,2015,02/10/2018 03:50:01 PM,41.800802,-87.622619,"(41.800802415, -87.622619343)"
4,10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,...,28.0,25.0,03,1144920.0,1898709.0,2015,02/10/2018 03:50:01 PM,41.878065,-87.743354,"(41.878064761, -87.743354013)"


In [4]:
crime.describe()

,ID,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year,Latitude,Longitude
count,6.802359e+06,6.802359e+06,6.802312e+06,6.187505e+06,6.188828e+06,6.740394e+06,6.740394e+06,6.802359e+06,6.740394e+06,6.740394e+06
mean,6.267879e+06,1.191551e+03,1.130251e+01,2.268313e+01,3.758620e+01,1.164517e+06,1.885721e+06,2.008401e+03,4.184201e+01,-8.767180e+01
std,3.063983e+06,7.033569e+02,6.945669e+00,1.383246e+01,2.154045e+01,1.716683e+04,3.270468e+04,5.067564e+00,8.999481e-02,6.212430e-02
min,6.340000e+02,1.110000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.001000e+03,3.661945e+01,-9.168657e+01
25%,3.443594e+06,6.220000e+02,6.000000e+00,1.000000e+01,2.300000e+01,1.152934e+06,1.859189e+06,2.004000e+03,4.176892e+01,-8.771387e+01
50%,6.254311e+06,1.111000e+03,1.000000e+01,2.200000e+01,3.200000e+01,1.165991e+06,1.890586e+06,2.008000e+03,4.185537e+01,-8.766618e+01
75%,8.909085e+06,1.731000e+03,1.700000e+01,3.400000e+01,5.800000e+01,1.176352e+06,1.909313e+06,2.012000e+03,4.190686e+01,-8.762836e+01
max,1.158934e+07,2.535000e+03,3.100000e+01,5.000000e+01,7.700000e+01,1.205119e+06,1.951622e+06,2.019000e+03,4.202291e+01,-8.752453e+01


In [5]:
print('Total Number or Crimes:\t\t\t{0}'.format(len(crime)))
print('Total number of Unique Districts:\t{0}'.format(len(crime['District'].unique())))

Total Number or Crimes:			6802359
Total number of Unique Districts:	25


Some crimes appear more than others. Later on, we might have to select the top crimes to analyze, so here they are

In [6]:
crime['Primary Type'].value_counts().nlargest(3)

THEFT              1431614
BATTERY            1241949
CRIMINAL DAMAGE     776795
Name: Primary Type, dtype: int64

# Part 3 - Data Cleaning and Preprocessing

This data is fairly clean, because it is maintained by the Chicago Police Department, which has high organizational standards. There aren't very many invalid crimes or unknowns.

To clean the data, we will do the following (not necessarily in order):

1. Drop useless columns
2. Standardize the location description
3. Standardize the crime type to get rid of the uppercase, standardize the non-criminal type
4. Convert True to 1.0 and False to 0.0
5. Remove NaN values

In [7]:
# first, drop the location description column and community area column because 
crime = crime.drop(['Location Description', 'Block', 'Community Area','Latitude', 'Beat', 'Ward','Longitude','X Coordinate','Y Coordinate','Location','Ward'],axis=1)

In [8]:
"""
    Author: James McDougall
    Param: string - is the string which is the name of the Primary Type
    Returns: a variable of type str which is lower case and represents  a more standardized type
"""
def standardize_primary_type(string):
    # compile all non-criminal offenses into on label
    if string == 'NON-CRIMINAL (SUBJECT SPECIFIED)' or string == 'NON - CRIMINAL' or string == 'NON-CRIMINAL':
        return 'non-criminal'
    if string == 'OTHER OFFENSE':
        return 'other'
    # rename crim sexual assault to just sexual assault to make it easier to read
    if string == 'CRIM SEXUAL ASSAULT':
        return 'sexual assault'
    else:
        # everything else, make sure to lowercase it so we don't have to use caps lock lol
        return string.lower()
        

Removing NaNs. Check that the two important columns don't have NaNs.

In [9]:
to_drop = crime[ crime['District'].isnull()]
print('Number of rows to drop with NaN district: ' + str(len(to_drop)))
crime = crime.drop(to_drop.index,axis=0)

to_drop = crime[crime['Primary Type'].isnull()]
print('Number of row to drop with NaN Primary Type: ' + str(len(to_drop)))
crime = crime.drop(to_drop.index,axis=0)

Number of rows to drop with NaN district: 47
Number of row to drop with NaN Primary Type: 0


To standardize the data we do the following:
1. standardize the type to be all lower case, merge some similar crimes that are formatted differently together
2. convert district, which is a float, to a string, because district should be categorical, not numerical.

In [10]:
crime['Type'] = crime['Primary Type'].apply(standardize_primary_type)
crime['District'] = crime['District'].astype(str)

def arrest_to_int(string):
    if string == True:
        return 1
    elif string == False:
        return 0
crimes['Arrest'] = crimes['Arrest'].apply(arrest_to_int)

# Part 4 - Data Visualization

# Part 5 - Data Analysis and Results

# Part 6 - Privacy/Ethics Considerations

The data is private because there is no identifying information. There are no names, only crime ids and description. The only identifying information is the location. This could be an issue because some of the crimes occurred in apartments, so you could conceivably find out the location where a crime was committed and use this to find someone.

A potential ethics considerations involving this data is that we are publicizing the locations of high crime, which could be used to discriminate against communities. While this data is publicily available, we are organizing and formatting it in a way that can be used to show which districts have a high crime rate. People with ill intentions could use this to argue that no one should go near these areas, alientating those communities.

# Part 7 - Conclusions and Discussion